In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/Chapter6/'

/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/Chapter6


In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

In [5]:
sc =SparkContext()

In [6]:
spark = SparkSession(sc)

In [7]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True,header=True)

In [8]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
print((data.count(), len(data.columns)))

(500, 8)


In [10]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [11]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

## Vector Assembler

In [15]:
# Create an assembler
assembler = VectorAssembler(
                            inputCols=['Avg Session Length', 'Time on App',
                                       'Time on Website', 'Length of Membership'],
                            outputCol='features'
                            ) # inputs

In [16]:
# Transfom data by assembler
data_pre = assembler.transform(data)

In [17]:
# Data_pre after transfoming
data_pre.select('features').show(2,truncate=False)

+--------------------------------------------------------------------------+
|features                                                                  |
+--------------------------------------------------------------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]|
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]|
+--------------------------------------------------------------------------+
only showing top 2 rows



In [19]:
data_pre.show(2,False)

+-------------------------+--------------------------------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------------------------------------------------------------+
|Email                    |Address                                     |Avatar   |Avg Session Length|Time on App       |Time on Website   |Length of Membership|Yearly Amount Spent|features                                                                  |
+-------------------------+--------------------------------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------------------------------------------------------------+
|mstephenson@fernandez.com|835 Frank TunnelWrightmouth, MI 82180-9605  |Violet   |34.49726772511229 |12.65565114916675 |39.57766801952616 |4.0826206329529615  |587.9510539684005  |[34.49726772511229,12.65565114916675,39.577668019526

=> Just one vectors instead of 4 columns

In [18]:
# Just select features (after transforming from 4 columns) and output
final_data = data_pre.select('features','Yearly Amount Spent')

In [21]:
final_data.show(2,truncate=False)

+--------------------------------------------------------------------------+-------------------+
|features                                                                  |Yearly Amount Spent|
+--------------------------------------------------------------------------+-------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]|587.9510539684005  |
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]|392.2049334443264  |
+--------------------------------------------------------------------------+-------------------+
only showing top 2 rows



## Train, Test split

In [23]:
# Train, Test split with ratio: 70:30
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean| 500.57401418286264|
| stddev|  75.12236656032736|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean|  496.4019084733527|
| stddev|  88.44580636768417|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



=> Train data and test data description are the same. Do not have an significant difference

## Build model

In [27]:
# Create a Linear Regression Model object
lr = LinearRegression(featuresCol='features',
                      labelCol='Yearly Amount Spent',
                      predictionCol='Predict_Yearly Amount Spent')
# => default: featuresCol='features', labelCol='label', predictionCol='prediction'

In [28]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [29]:
# Print the cofficients and intercept for linear regression
print('Coefficients : {} Intercept: {}'.format(lrModel.coefficients, lrModel.intercept))

Coefficients : [25.65684909228494,38.87472553563457,0.7894175450095725,61.41846187187868] Intercept: -1063.9228999355619


## Model evaluation

In [30]:
test_results = lrModel.evaluate(test_data)

In [31]:
# Interesting results...
test_results.residuals.show(5)

+------------------+
|         residuals|
+------------------+
| 7.533977361789539|
| 3.456838996569445|
|18.570630306753912|
|-6.173747902310652|
|-4.883619251649009|
+------------------+
only showing top 5 rows



In [35]:
print('RMSE: {}'.format(test_results.rootMeanSquaredError))
print('MSE: {}'.format(test_results.meanSquaredError))
print('r2: {}'.format(test_results.r2))

RMSE: 9.396456328064694
MSE: 88.29339152522702
r2: 0.988637879367043


In [36]:
# check test dataset
test_model = lrModel.transform(test_data)

In [38]:
# Inspect results
test_model.select('Predict_Yearly Amount Spent', 'Yearly Amount Spent').show(5)

+---------------------------+-------------------+
|Predict_Yearly Amount Spent|Yearly Amount Spent|
+---------------------------+-------------------+
|          487.1046323951032|  494.6386097568927|
|          429.2638788433642|  432.7207178399336|
|          445.0207877211867|  463.5914180279406|
|         495.98623589877207|  489.8124879964614|
|          280.8020399020347|  275.9184206503857|
+---------------------------+-------------------+
only showing top 5 rows



## Save and Load model

In [39]:
# Save model
lrModel.save('lrModel_Ecommerce_Customers')

In [41]:
from pyspark.ml.regression import LinearRegressionModel

In [42]:
# Load model
lrModel2 = LinearRegressionModel.load('lrModel_Ecommerce_Customers')

## New data prediction

In [43]:
# Predict new values (assuming select test_data)
unlabeled_data = test_data.select('features')

In [44]:
predictions = lrModel2.transform(unlabeled_data)

In [45]:
predictions.show(5)

+--------------------+---------------------------+
|            features|Predict_Yearly Amount Spent|
+--------------------+---------------------------+
|[30.9716756438877...|          487.1046323951032|
|[31.3091926408918...|          429.2638788433642|
|[31.3123495994443...|          445.0207877211867|
|[31.5147378578019...|         495.98623589877207|
|[31.5171218025062...|          280.8020399020347|
+--------------------+---------------------------+
only showing top 5 rows



=> In fact, we have a new data and have to standardize and transform before predicting